In [1]:
pip install rioxarray


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install numpy gdal==$(gdal-config --version)


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install scipy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install matplotlib


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install geopandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
from osgeo import gdal
import xarray as xr
import rioxarray as rxr
import os
from glob import glob
from datetime import datetime, timedelta
import numpy as np
import calendar
import numpy as n
import rasterio

In [ ]:

#TODO: This data gets resutls for entire india and daily temporal resolution

def convert_grd_to_tif(grd_file, output_dir):
    try:
        # Extract year from filename (assuming format YYYY.grd)
        year = os.path.basename(grd_file).split('.')[0]
        print(f"Processing year: {year}")
        
        # Open the .grd file and read its content
        with open(grd_file, 'rb') as f:
            data = f.read()
        print(f"Read {len(data)} bytes from {grd_file}")
        
        # Determine the number of days in the year
        is_leap_year = (int(year) % 4 == 0 and int(year) % 100 != 0) or (int(year) % 400 == 0)
        days_in_year = 366 if is_leap_year else 365
        print(f"Year {year} has {days_in_year} days")
        
        # Convert binary data to numpy array
        all_data = np.frombuffer(data, dtype=np.float32)
        
        # For 0.25° x 0.25° grid covering India
        grid_size = 129 * 135  # Total number of grid cells
        
        for day in range(days_in_year):
            date = datetime(int(year), 1, 1) + timedelta(days=day)
            output_filename = f"imd_25_{date.strftime('%Y-%m-%d')}.tif"
            output_file = os.path.join(output_dir, output_filename)
            
            # Extract data for the current day
            day_data = all_data[day * grid_size : (day + 1) * grid_size].reshape(129, 135)
            
            # Flip the data vertically
            day_data = np.flipud(day_data)
            
            # Create a new GeoTIFF file
            driver = gdal.GetDriverByName('GTiff')
            dataset = driver.Create(output_file, 135, 129, 1, gdal.GDT_Float32)
            
            # Set geotransform
            geotransform = (65.0, 0.25, 0, 6.0, 0, 0.25)
            dataset.SetGeoTransform(geotransform)
            
            # Set projection (assuming WGS84)
            srs = gdal.osr.SpatialReference()
            srs.ImportFromEPSG(4326)  # WGS84
            dataset.SetProjection(srs.ExportToWkt())
            
            # Write the data
            dataset.GetRasterBand(1).WriteArray(day_data)
            
            # Close the dataset
            dataset = None
            
            if os.path.exists(output_file):
                print(f"Successfully saved {output_file} (size: {os.path.getsize(output_file)} bytes)")
            else:
                print(f"Failed to create {output_file}")
            
    except Exception as e:
        print(f"Failed to process {grd_file}: {e}")
        # Define the directory containing the rainfall .grd files
rainfall_dir =r'~/dev/btech-project/data/rain_grd'

# Define the output directory for the .tif files
output_dir = r'~/dev/btech-project/data/rain_tif'
os.makedirs(output_dir, exist_ok=True)

# List all .grd files in the rainfall directory
grd_files = glob(os.path.join(rainfall_dir, '*.grd'))

if not grd_files:
    print(f"No .grd files found in {rainfall_dir}")
else:
    for grd_file in grd_files:
        print(f"Processing {grd_file}...")
        convert_grd_to_tif(grd_file, output_dir)

print("Script execution completed.")




In [2]:
import os
from datetime import datetime, timedelta
from glob import glob
import numpy as np
from osgeo import gdal, osr
import warnings
warnings.filterwarnings('ignore')

# def get_southern_india_bounds():
#     """Define the bounding box for Southern India covering Western Ghats"""
#     return {
#         'min_lon': 74.0,  # Western boundary
#         'max_lon': 78.5,  # Eastern boundary
#         'min_lat': 8.0,   # Southern boundary
#         'max_lat': 15.0   # Northern boundary
#     }
    
def get_delhi_ncr_bounds():
    """Define the bounding box for Delhi NCR region"""
    return {
        'min_lon': 76.8,  # Western boundary (covers parts of Gurgaon)
        'max_lon': 77.6,  # Eastern boundary (covers parts of Ghaziabad/Noida)
        'min_lat': 28.4,  # Southern boundary 
        'max_lat': 28.9   # Northern boundary
    }

def convert_grd_to_clipped_tif(grd_file, output_dir):
    """Convert GRD file to daily GeoTIFF files clipped to Delhi NCR region"""
    try:
        # Extract year from filename (assuming format YYYY.grd)
        year = os.path.basename(grd_file).split('.')[0]
        print(f"Processing year: {year}")
        
        # Open the .grd file and read its content
        with open(grd_file, 'rb') as f:
            data = f.read()
        print(f"Read {len(data)} bytes from {grd_file}")
        
        # Determine the number of days in the year
        is_leap_year = (int(year) % 4 == 0 and int(year) % 100 != 0) or (int(year) % 400 == 0)
        days_in_year = 366 if is_leap_year else 365
        print(f"Year {year} has {days_in_year} days")
        
        # Convert binary data to numpy array
        all_data = np.frombuffer(data, dtype=np.float32)
        
        # For 0.25° x 0.25° grid covering India
        grid_size = 129 * 135  # Total number of grid cells
        
        # Get Delhi NCR bounds
        bounds = get_delhi_ncr_bounds()
        
        # Calculate array indices for clipping
        min_x = int((bounds['min_lon'] - 65.0) / 0.25)
        max_x = int((bounds['max_lon'] - 65.0) / 0.25)
        min_y = int((40.0 - bounds['max_lat']) / 0.25)  # Flip Y coordinates
        max_y = int((40.0 - bounds['min_lat']) / 0.25)
        
        # Calculate new dimensions
        new_width = max_x - min_x + 1
        new_height = max_y - min_y + 1
        
        for day in range(days_in_year):
            date = datetime(int(year), 1, 1) + timedelta(days=day)
            output_filename = f"imd_25_south_{date.strftime('%Y-%m-%d')}.tif"
            output_file = os.path.join(output_dir, output_filename)
            
            # Extract data for the current day
            day_data = all_data[day * grid_size : (day + 1) * grid_size].reshape(129, 135)
            
            # Flip the data vertically
            day_data = np.flipud(day_data)
            
            # Clip the data to Southern India region
            clipped_data = day_data[min_y:max_y+1, min_x:max_x+1]
            
            # Create a new GeoTIFF file
            driver = gdal.GetDriverByName('GTiff')
            dataset = driver.Create(output_file, new_width, new_height, 1, gdal.GDT_Float32)
            
            # Set geotransform for the clipped region
            geotransform = (
                bounds['min_lon'],  # Left edge
                0.25,               # Pixel width
                0,                  # Rotation (0 if image is "north up")
                bounds['max_lat'],  # Top edge
                0,                  # Rotation (0 if image is "north up")
                -0.25              # Pixel height (negative as it goes from top to bottom)
            )
            dataset.SetGeoTransform(geotransform)
            
            # Set projection (WGS84)
            srs = osr.SpatialReference()
            srs.ImportFromEPSG(4326)
            dataset.SetProjection(srs.ExportToWkt())
            
            # Write the clipped data
            dataset.GetRasterBand(1).WriteArray(clipped_data)
            
            # Set NoData value
            dataset.GetRasterBand(1).SetNoDataValue(-999)
            
            # Close the dataset
            dataset = None
            
            if os.path.exists(output_file):
                print(f"Successfully saved {output_file} (size: {os.path.getsize(output_file)} bytes)")
            else:
                print(f"Failed to create {output_file}")
            
    except Exception as e:
        print(f"Failed to process {grd_file}: {str(e)}")
        raise

def main():
    # Define the directory containing the rainfall .grd files
    rainfall_dir = r'~/dev/btech-project/data/rain_grd'
    
    # Define the output directory for the clipped .tif files
    output_dir = r'~/dev/btech-project/data/rain_tif'
    os.makedirs(output_dir, exist_ok=True)
    
    # List all .grd files in the rainfall directory
    grd_files = glob(os.path.join(rainfall_dir, '*.grd'))
    
    if not grd_files:
        print(f"No .grd files found in {rainfall_dir}")
    else:
        for grd_file in grd_files:
            print(f"Processing {grd_file}...")
            convert_grd_to_clipped_tif(grd_file, output_dir)
    
    print("Script execution completed.")

if __name__ == "__main__":
    main()

No .grd files found in ~/dev/btech-project/data/rain_grd
Script execution completed.


In [ ]:
def convert_grd_to_tif_temp(grd_file, output_dir):
    try:
        # Extract year from filename (assuming format YYYY.grd)
        year = os.path.basename(grd_file).split('.')[0]
        print(f"Processing year: {year}")
        
        # Open the .grd file and read its content
        with open(grd_file, 'rb') as f:
            data = f.read()
        print(f"Read {len(data)} bytes from {grd_file}")
        
        # Determine the number of days in the year
        is_leap_year = (int(year) % 4 == 0 and int(year) % 100 != 0) or (int(year) % 400 == 0)
        days_in_year = 366 if is_leap_year else 365
        print(f"Year {year} has {days_in_year} days")
        
        # Convert binary data to numpy array
        all_data = np.frombuffer(data, dtype=np.float32)
        
        # For 1° x 1° grid covering India (adjust these values if needed)
        grid_size = 31 * 31  # Total number of grid cells
        
        for day in range(days_in_year):
            date = datetime(int(year), 1, 1) + timedelta(days=day)
            output_filename = f"imd_1.00_{date.strftime('%Y-%m-%d')}_temp.tif"
            output_file = os.path.join(output_dir, output_filename)
            
            # Extract data for the current day
            day_data = all_data[day * grid_size : (day + 1) * grid_size].reshape(31, 31)
            
            # Flip the data vertically
            day_data = np.flipud(day_data)
            
            # Create a new GeoTIFF file
            driver = gdal.GetDriverByName('GTiff')
            dataset = driver.Create(output_file, 31, 31, 1, gdal.GDT_Float32)
            
            # Set geotransform (adjust these values if needed)
            geotransform = (67.0, 1.0, 0, 8.0, 0, 1.0)
            dataset.SetGeoTransform(geotransform)
            
            # Set projection (assuming WGS84)
            srs = gdal.osr.SpatialReference()
            srs.ImportFromEPSG(4326)  # WGS84
            dataset.SetProjection(srs.ExportToWkt())
            
            # Write the data
            dataset.GetRasterBand(1).WriteArray(day_data)
            
            # Close the dataset
            dataset = None
            
            if os.path.exists(output_file):
                print(f"Successfully saved {output_file} (size: {os.path.getsize(output_file)} bytes)")
            else:
                print(f"Failed to create {output_file}")
            
    except Exception as e:
        print(f"Failed to process {grd_file}: {e}")

# Define the directory containing the temperature .grd files
temp_dir = r'C:\Users\admin\Downloads\tmax_grd'
# Define the output directory for the .tif files
output_dir = r'D:\tif_tmax'
os.makedirs(output_dir, exist_ok=True)

# List all .grd files in the temperature directory
grd_files = glob(os.path.join(temp_dir, '*.grd'))
if not grd_files:
    print(f"No .grd files found in {temp_dir}")
else:
    for grd_file in grd_files:
        print(f"Processing {grd_file}...")
        convert_grd_to_tif_temp(grd_file, output_dir)

print("Script execution completed.")




In [ ]:
def convert_grd_to_tif_temp(grd_file, output_dir):
    try:
        # Extract year from filename (assuming format YYYY.grd)
        year = os.path.basename(grd_file).split('.')[0]
        print(f"Processing year: {year}")
        
        # Open the .grd file and read its content
        with open(grd_file, 'rb') as f:
            data = f.read()
        print(f"Read {len(data)} bytes from {grd_file}")
        
        # Determine the number of days in the year
        is_leap_year = (int(year) % 4 == 0 and int(year) % 100 != 0) or (int(year) % 400 == 0)
        days_in_year = 366 if is_leap_year else 365
        print(f"Year {year} has {days_in_year} days")
        
        # Convert binary data to numpy array
        all_data = np.frombuffer(data, dtype=np.float32)
        
        # For 1° x 1° grid covering India (adjust these values if needed)
        grid_size = 31 * 31  # Total number of grid cells
        
        for day in range(days_in_year):
            date = datetime(int(year), 1, 1) + timedelta(days=day)
            output_filename = f"imd_1.00_{date.strftime('%Y-%m-%d')}_temp.tif"
            output_file = os.path.join(output_dir, output_filename)
            
            # Extract data for the current day
            day_data = all_data[day * grid_size : (day + 1) * grid_size].reshape(31, 31)
            
            # Flip the data vertically
            day_data = np.flipud(day_data)
            
            # Create a new GeoTIFF file
            driver = gdal.GetDriverByName('GTiff')
            dataset = driver.Create(output_file, 31, 31, 1, gdal.GDT_Float32)
            
            # Set geotransform (adjust these values if needed)
            geotransform = (67.0, 1.0, 0, 8.0, 0, 1.0)
            dataset.SetGeoTransform(geotransform)
            
            # Set projection (assuming WGS84)
            srs = gdal.osr.SpatialReference()
            srs.ImportFromEPSG(4326)  # WGS84
            dataset.SetProjection(srs.ExportToWkt())
            
            # Write the data
            dataset.GetRasterBand(1).WriteArray(day_data)
            
            # Close the dataset
            dataset = None
            
            if os.path.exists(output_file):
                print(f"Successfully saved {output_file} (size: {os.path.getsize(output_file)} bytes)")
            else:
                print(f"Failed to create {output_file}")
            
    except Exception as e:
        print(f"Failed to process {grd_file}: {e}")

# Define the directory containing the temperature .grd files
temp_dir = r'C:\Users\admin\Downloads\tmin_grd'
# Define the output directory for the .tif files
output_dir = r'D:\tif_tmin'
os.makedirs(output_dir, exist_ok=True)

# List all .grd files in the temperature directory
grd_files = glob(os.path.join(temp_dir, '*.grd'))
if not grd_files:
    print(f"No .grd files found in {temp_dir}")
else:
    for grd_file in grd_files:
        print(f"Processing {grd_file}...")
        convert_grd_to_tif_temp(grd_file, output_dir)

print("Script execution completed.")

# Check the output directory
print("\nChecking output directory:")
output_files = glob(os.path.join(output_dir, '*.tif'))
if output_files:
    print(f"Found {len(output_files)} .tif files in the output directory.")
    for file in output_files[:5]:  # Print first 5 files
        print(f" - {os.path.basename(file)} (size: {os.path.getsize(file)} bytes)")
    if len(output_files) > 5:
        print(f" ... and {len(output_files) - 5} more.")
else:
    print("No .tif files found in the output directory.")





In [ ]:
import os
import numpy as np
import calendar
from datetime import datetime
from glob import glob
from osgeo import gdal

def calculate_monthly_rainfall(input_dir, output_dir, year):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Dictionary to store monthly data
    monthly_data = {month: [] for month in range(1, 13)}
    
    # Read all daily files for the specified year
    daily_files = sorted(glob(os.path.join(input_dir, f'imd_0.25_{year}-*.tif')))
    
    for file in daily_files:
        # Extract date from filename
        date_str = os.path.basename(file).split('_')[2].replace('.tif', '')
        date = datetime.strptime(date_str, '%Y-%m-%d')
        
        # Read the raster data
        ds = gdal.Open(file)
        band = ds.GetRasterBand(1)
        data = band.ReadAsArray()
        
        # Store the data in the corresponding month's list
        monthly_data[date.month].append(data)
        
        # Close the dataset
        ds = None
    
    # Get geotransform and projection from a sample daily file
    if daily_files:
        sample_ds = gdal.Open(daily_files[0])
        geotransform = sample_ds.GetGeoTransform()
        projection = sample_ds.GetProjection()
        sample_ds = None
    
        # Calculate monthly totals and save as new .tif files
        for month in range(1, 13):
            if monthly_data[month]:  # Check if there's data for this month
                # Calculate the sum
                monthly_sum = np.sum(monthly_data[month], axis=0)
                
                # Create a new GeoTIFF file for the monthly sum
                output_file = os.path.join(output_dir, f'{year}_{month:02d}_{calendar.month_abbr[month]}_total_rainfall.tif')
                
                # Create the output file
                driver = gdal.GetDriverByName('GTiff')
                out_ds = driver.Create(output_file, monthly_sum.shape[1], monthly_sum.shape[0], 1, gdal.GDT_Float32)
                out_ds.SetGeoTransform(geotransform)
                out_ds.SetProjection(projection)
                
                # Write the data
                out_band = out_ds.GetRasterBand(1)
                out_band.WriteArray(monthly_sum)
                
                # Close the dataset
                out_ds = None
                
                print(f"Saved total rainfall for {calendar.month_name[month]} {year} to {output_file}")
            else:
                print(f"No data found for {calendar.month_name[month]} {year}")
    else:
        print(f"No daily files found for the year {year}")

# Set your input and output directories
input_dir = r'D:\selected_tif'
output_dir = r'D:\monthly_2018_rainfall'
# Set the year you want to process
year_to_process = 2018

# Run the function
calculate_monthly_rainfall(input_dir, output_dir, year_to_process)
print("Monthly rainfall calculation completed.")
